In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
full_image_a = browser.find_by_id('full_image')
full_image_a.click()
browser.is_element_present_by_text('more info',wait_time=1)
more_info_a = browser.find_link_by_partial_text('more info')
more_info_a.click()
html = browser.html
soup = bs(html, 'html.parser')
# featured_image_url = soup.select_one("article.carousel_item").get("style")
# featured_image_url.split('(')[1].split("'")[1]
featured_image_url = soup.select_one('figure.lede a img').get('src')
featured_image_url

'/spaceimages/images/largesize/PIA16726_hires.jpg'

In [4]:
image_url = f'https://jpl.nasa.gov{featured_image_url}'
image_url

'https://jpl.nasa.gov/spaceimages/images/largesize/PIA16726_hires.jpg'

In [6]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
slide = soup.select_one('ul.item_list li.slide')
news_title = slide.find('div', class_="content_title").get_text()
news_title

"NASA's Briefcase-Size MarCO Satellite Picks Up Honors"

In [7]:
news_p = slide.find('div', class_="article_teaser_body").text
news_p

'The twin spacecraft, the first of their kind to fly into deep space, earn a Laureate from Aviation Week & Space Technology.'

In [8]:
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)
soup = bs(response.text, 'html.parser')
#tweet = soup.find('div',attrs={"class":"tweet","data-name":"Mars Weather"})
# mars_weather = tweet.find('p','tweet-text').text
# mars_weather
tweet = soup.find('div',class_='js-tweet-text-container').text
mars_weather = tweet.translate({ord(c): None for c in '\n'})
mars_weather

'InSight sol 360 (2019-12-01) low -100.1ºC (-148.3ºF) high -20.6ºC (-5.1ºF)winds from the SSE at 5.0 m/s (11.2 mph) gusting to 19.5 m/s (43.7 mph)pressure at 6.70 hPapic.twitter.com/YLEhj8gQGi'

In [24]:
url = 'https://space-facts.com/mars/'  
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser') 
table = soup.find('table', attrs={"class":"tablepress tablepress-id-p-mars"})
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
table_df = pd.DataFrame(l, columns=["description", "value"])
table_df.set_index("description",inplace=True)
table_df
table_df.to_html(classes="table table-striped")

'<table border="1" class="dataframe table table-striped">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronom

In [10]:
equatorial_diameter = table_df.loc[0,'B']  
polar_diameter = table_df.loc[1,'B']  
mass = table_df.loc[2,'B']  
moons = table_df.loc[3,'B']  
orbit_distance = table_df.loc[4,'B']  
equatorial_diameter

'6,792 km'

In [11]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars' 
browser.visit(url)

hemispheres = ['Cerberus Hemisphere', 'Valles Marineris Hemisphere', 'Syrtis Major Hemisphere', 'Schiaparelli Hemisphere']
images = ['cerberus_enhanced', "valles_marineris_enhanced", 'syrtis_major_enhanced', 'schiaparelli_enhanced']
hemisphere_image_urls = []

for h, i in zip(hemispheres, images):
    hemisphere_a = browser.find_link_by_partial_text(h)
    hemisphere_a.click()
    html = browser.html
    soup = bs(html, 'html.parser')
    img = soup.find('img',class_='wide-image').get('src')
#     for i in images:
    img_url = f'https://astrogeology.usgs.gov/search/map/Mars/Viking/{i}{img}'
    title = soup.find("h2",class_="title").get_text()
    hemisphere_image_urls.append( {"title":title, "img_url": img_url})
#     i = i + 1
for title in hemisphere_image_urls:
    print(f'{title}, {img_url}')
#   print(hemisphere_image_urls[title], hemisphere_image_urls[img_url])
# urls_df = pd.DataFrame(hemisphere_image_urls, columns=['A', 'B'])
# urls_df.head()

#             hemisphere_image_urls = [
#                  {'title': title, 'img_url': img_url}
#              ]
#              #db.items.insert_one(hemisphere_image_urls)



{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced/cache/images/0eb84c5fa8ded69d32e9f213497a1960_schiaparelli_unenhanced.tif_full.jpg
{'title': 'Valles Marineris Hemisphere Unenhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced/cache/images/8981a6d7057a7634dc312c0448f462c1_valles_marineris_unenhanced.tif_full.jpg'}, https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced/cache/images/0eb84c5fa8ded69d32e9f213497a1960_schiaparelli_unenhanced.tif_full.jpg
{'title': 'Syrtis Major Hemisphere Unenhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced/cache/images/4951731d09d453e340fccf70e43c5933_syrtis_major_unenhanced.tif_full.jpg'}, https://astrogeology.usgs